<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# TL;DR

1. In this lab scenario you will have a chance to compare performance of the classic RNN and LSTM on a toy example.
2. This toy example will show that maintaining memory over even 20 steps is non-trivial.
3. Finally, you will see how curriculum learning may allow to train a model on larger sequences.

# Problem definition

Here we consider a toy example, where the goal is to discriminate between two types of binary sequences:
* [Type 0] a sequence with exactly one zero (remaining entries are equal to one).
* [Type 1] a sequence full of ones,

We are especially interested in the performance of the trained models on discriminating between a sequence full of ones versus a sequence with leading zero followed by ones. Note that in this case the goal of the model is to output the first element of the sequence, as the label (sequence type) is fully determined by the first element of the sequence.

#Implementation

## Importing torch

Install `torch` and `torchvision`

In [1]:
!pip3 install torch torchvision


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from typing import List
from tqdm import tqdm

torch.manual_seed(1)

## Understand dimensionality

Check the input and output specification [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) and [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). The following snippet shows how we can process
a sequence by LSTM and output a vector of size `hidden_dim` after reading
each token of the sequence.

In [3]:
hidden_dim = 5
lstm = nn.LSTM(1, hidden_dim)  # Input sequence contains elements - vectors of size 1

# create a random sequence
sequence = [torch.randn(1) for _ in range(10)]

# initialize the hidden state (including cell state)
hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

for i, elem in enumerate(sequence):
  # we are processing only a single element of the sequence, and there
  # is only one sample (sequence) in the batch, the third one
  # corresponds to the fact that our sequence contains elemenents,
  # which can be treated as vectors of size 1
  out, hidden = lstm(elem.view(1, 1, 1), hidden)
  print(f'i={i} out={out.detach()}')
print(f'Final hidden state={hidden[0].detach()} cell state={hidden[1].detach()}')

i=0 out=tensor([[[-0.0675,  0.1179,  0.1081,  0.0414, -0.0341]]])
i=1 out=tensor([[[-0.1067,  0.1726,  0.1400,  0.0902, -0.0596]]])
i=2 out=tensor([[[-0.1148,  0.1885,  0.1956,  0.0974, -0.0840]]])
i=3 out=tensor([[[-0.1270,  0.2031,  0.1495,  0.1249, -0.0860]]])
i=4 out=tensor([[[-0.1281,  0.2019,  0.1810,  0.1475, -0.1027]]])
i=5 out=tensor([[[-0.1274,  0.2060,  0.0798,  0.1330, -0.0860]]])
i=6 out=tensor([[[-0.1318,  0.2039,  0.0997,  0.1772, -0.1011]]])
i=7 out=tensor([[[-0.1145,  0.2008, -0.0431,  0.1051, -0.0717]]])
i=8 out=tensor([[[-0.1289,  0.1989,  0.0515,  0.1944, -0.1030]]])
i=9 out=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]])
Final hidden state=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]]) cell state=tensor([[[-0.2590,  0.4080,  0.1307,  0.4329, -0.2895]]])


## To implement

Process the whole sequence all at once by calling `lstm` only once and check that the output is exactly the same as above (remember to initialize the hidden state the same way).

In [4]:
# #########################################################
#                    To implement
# #########################################################

hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

out, hidden = lstm(torch.stack(sequence).view(len(sequence), 1, 1), hidden)

for i in range(len(sequence)):
  print(f'i={i} out={out[i].detach()}')
print(f'Final hidden state={hidden[0].detach()} cell state={hidden[1].detach()}')

i=0 out=tensor([[-0.0675,  0.1179,  0.1081,  0.0414, -0.0341]])
i=1 out=tensor([[-0.1067,  0.1726,  0.1400,  0.0902, -0.0596]])
i=2 out=tensor([[-0.1148,  0.1885,  0.1956,  0.0974, -0.0840]])
i=3 out=tensor([[-0.1270,  0.2031,  0.1495,  0.1249, -0.0860]])
i=4 out=tensor([[-0.1281,  0.2019,  0.1810,  0.1475, -0.1027]])
i=5 out=tensor([[-0.1274,  0.2060,  0.0798,  0.1330, -0.0860]])
i=6 out=tensor([[-0.1318,  0.2039,  0.0997,  0.1772, -0.1011]])
i=7 out=tensor([[-0.1145,  0.2008, -0.0431,  0.1051, -0.0717]])
i=8 out=tensor([[-0.1289,  0.1989,  0.0515,  0.1944, -0.1030]])
i=9 out=tensor([[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]])
Final hidden state=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]]) cell state=tensor([[[-0.2590,  0.4080,  0.1307,  0.4329, -0.2895]]])


## Training a model

Below we define a very simple model, which is a single layer of LSTM, where the output in each time step is processed by relu followed by a single fully connected layer, the output of which is a single number. We are going
to use the number generated after reading the last element of the sequence,
which will serve as the logit for our classification problem.

In [5]:
class Model(nn.Module):

    def __init__(self, hidden_dim: int):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.lstm(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

Below is a training loop, where we only train on the two hardest examples.

In [6]:
SEQUENCE_LEN = 10

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]


def eval_on_hard_examples(model: nn.Module) -> List[float]:
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')
        return logits


def train_model(hidden_dim: int, lr: float, num_steps:int = 10000) -> None:
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    pbar = tqdm(range(num_steps))
    for step in pbar:
        if step % 100 == 0:
            logits = eval_on_hard_examples(model)
            pbar.set_postfix(logits=logits)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))

            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()

In [7]:
train_model(hidden_dim=20, lr=0.01, num_steps=10000)

  1%|          | 70/10000 [00:00<00:27, 362.99it/s, logits=[tensor([-0.1775]), tensor([-0.1774])]]

Logits for hard examples=[tensor([-0.1775]), tensor([-0.1774])]


  2%|▏         | 154/10000 [00:00<00:24, 399.33it/s, logits=[tensor([0.0095]), tensor([0.0097])]] 

Logits for hard examples=[tensor([0.0095]), tensor([0.0097])]


  3%|▎         | 280/10000 [00:00<00:23, 413.26it/s, logits=[tensor([0.0259]), tensor([0.0263])]]

...

 99%|█████████▊| 9854/10000 [00:23<00:00, 415.50it/s, logits=[tensor([-12.5402]), tensor([12.3918])]]

Logits for hard examples=[tensor([-12.5402]), tensor([12.3918])]


 99%|█████████▉| 9939/10000 [00:24<00:00, 409.00it/s, logits=[tensor([-12.5423]), tensor([12.3946])]]

Logits for hard examples=[tensor([-12.5423]), tensor([12.3946])]


100%|██████████| 10000/10000 [00:24<00:00, 413.99it/s, logits=[tensor([-12.5423]), tensor([12.3946])]]


## To implement

1. Check for what values of `SEQUENCE_LEN` the model is able to discriminate betweeh the two hard examples (after training).
2. Instead of training on `HARD_EXAMPLES` only, modify the training loop to train on sequences where zero may be in any position of the sequence (so any valid sequence of `Type 0`, not just the hardest one). After modifying the training loop check for what values of `SEQUENCE_LEN` you can train the model successfully.
3. Replace LSTM by a classic RNN and check for what values of `SEQUENCE_LEN` you can train the model successfully.
4. Write a proper curricullum learning loop, where in a loop you consider longer and longer sequences, where expansion of the sequence length happens only after the model is trained successfully on the current length.

Note that for steps 2-4 you may need to change the value of `num_steps`.